In [ ]:
import multiprocessing as mp
import sys

import pandas as pd

import pfr

Below is the function processBS, which takes in a boxscore ID and returns a DataFrame of the raw (i.e., pre-feature engineering) play-by-play data provided by pro-football-reference.com (see, for example, [this page](http://www.pro-football-reference.com/boxscores/201010030pit.htm#pbp) for an example of play-by-play data). This is done using the ```pfr``` package for scraping data from [pro-football-reference.com](pro-football-reference.com). This package was written by our very own Matt Goldberg, and its code can be found [on his GitHub](github.com/mdgoldberg/pfr). Specifically, see the code in [pfr/boxscores.py](https://github.com/mdgoldberg/pfr/blob/master/pfr/boxscores.py) (and relevant code in [pfr/utils.py](https://github.com/mdgoldberg/pfr/blob/master/pfr/utils.py)) to see more about how this scraping is done.

In [ ]:
def processBS(bs):
    bs = pfr.boxscores.BoxScore(bs)
    addon = bs.pbp(keepErrors=True)
    return addon

The following function, getDataForYear, takes in a year as an argument and returns a DataFrame which concatenates the play-by-play DataFrames from every one of that season's games. This is done using python's [multiprocessing](https://docs.python.org/2/library/multiprocessing.html) module; using this, we can take advantage of the parallelizable nature of the task at hand.

In [ ]:
def getDataForYear(yr):
    bsIDs = set()
    for tmID in pfr.teams.listTeams():
        tm = pfr.teams.Team(tmID)
        bss = tm.boxscores(yr)
        bsIDs.update(bss)

    print len(bsIDs)
    pool = mp.Pool(mp.cpu_count())
    dfs = pool.map_async(processBS, bsIDs).get(sys.maxint)

    pbp = pd.concat(dfs)
    pbp = pbp.reset_index(drop=True)

    pbp.to_csv('{}plays.csv'.format(yr), index_share=False, index=False)

In [ ]:
for yr in xrange(2002, 2015):
    print yr
    getDataForYear(yr)